In [6]:
import os
import json
import asyncio
import websockets
import time
import pandas as pd
import numpy as np
import glob

import matplotlib.pyplot as plt
import moviepy.editor as mpy


from collections import deque

class LowPassFilter:
    def __init__(self, n):
        self.x_buffer = deque(maxlen=n)
        
    def lowpass(self, x):
        self.x_buffer.append(x)
        return sum(self.x_buffer) / len(self.x_buffer)

class PidController:
    def __init__(self, gains, set_point, max_measurement=1e6):
        self.gains = gains
        self.set_point = set_point
        self.max_measurement = max_measurement
        self.reset()
        
    def reset(self):
        self.history = []
        self.error = 0
        self.error_i = 0
        self.prev_measurement = None
        self.prev_error = 0
        self.sum_error_squared = 0
        self.steps = 0
        self.overflow_detected = False
        self.error_d_filter = LowPassFilter(5)
        
    
    def __call__(self, delta_t, measurement):
        self.error = self.set_point() - measurement
        
        self.error_i += self.error * delta_t
        error_d = (self.error - self.prev_error) / delta_t if self.steps > 0 else 0
        error_d = self.error_d_filter.lowpass(error_d)
            
        self.prev_error = self.error
        
        if (abs(measurement) > self.max_measurement):
            self.overflow_detected = True
            self.sum_error_squared += 1e8

        self.sum_error_squared += self.error**2
            
        self.steps += 1
        k_p, k_i, k_d = self.gains

        p_term = k_p * self.error
        i_term = k_i * self.error_i
        d_term = k_d * error_d
        
        output = p_term + i_term + d_term
        self.history.append((delta_t, measurement, self.error, self.error_i, error_d, p_term, i_term, d_term, output))
        return output
    
    def get_history(self):
        columns=['t','measurement', 'error', 'error_i', 'error_d', 'p_term', 'i_term', 'd_term', 'output']
        return pd.DataFrame(self.history, columns=columns)
    
    def get_error(self):
        return self.sum_error_squared / self.steps if self.steps > 0 else 0
    

class SimulatorResponder:
    valid_prefix = '42'
    start_delay = 150
    
    def __init__(self, throttle_controller, steering_controller):
        self.throttle_controller = throttle_controller
        self.steering_controller = steering_controller
        self.initialized = False
        self.step_count = 0
        self.prev_timestamp = 0

    def is_valid_message(self, message):
        return message.startswith(self.valid_prefix)
    
    def parse_message(self,message):
        return json.loads(message)
    
    def make_output_string(self, tag, payload):
        return self.valid_prefix + json.dumps([tag, payload])
                
    def step(self, timestamp, message):
        if not self.initialized:
            self.step_count = 0
            self.initialized = True
            return self.make_output_string('reset', {})

        self.step_count += 1
        delta_t = timestamp - self.prev_timestamp
        self.prev_timestamp = timestamp
        
        if self.step_count < self.start_delay:
            return self.make_output_string('manual', {})

        if self.is_valid_message(message):
            control = self.process(delta_t, message)
            return self.make_output_string('steer', control)
        else:
            return self.make_output_string('manual', {})
        
            
    def process(self, delta_t, message):
        tag, telemetry = self.parse_message(message[2:])
        if tag == 'telemetry':
            speed = float(telemetry['speed'])
            cte = float(telemetry['cte'])
            angle = float(telemetry['steering_angle'])
            
            steer_value = self.steering_controller(delta_t, cte)
            throttle_value = self.throttle_controller(delta_t, speed)
            output = {
                'steering_angle': steer_value,
                'throttle': throttle_value }

            return output


In [7]:
def run_simulator(throttle, steering, stop_condition):
    simulator = SimulatorResponder(throttle, steering)
    
    async def hello(websocket, path):
        throttle.reset()
        steering.reset()
        
        while not stop_condition(simulator):
            message = await websocket.recv()
            response = simulator.step(time.time(), message)
            await websocket.send(response)
        asyncio.get_event_loop().stop()
        

    start_server = websockets.serve(hello, 'localhost', 4567)
    asyncio.get_event_loop().run_until_complete(start_server)
    try:
        asyncio.get_event_loop().run_forever()
    finally:
        start_server.ws_server.close()
        start_server.ws_server.wait_closed()
    

def max_steps(limit):
    return lambda simulator: simulator.step_count > limit

def constant(x):
    return lambda: x

def proportional_speed(steering_controller, max_speed):
    return lambda: 60 - abs(steering_controller.error) * 50 / 3


In [8]:
def twiddle(simulation, initial_value, increment, tol=0.2, conv_factor=0.9): 
    p = initial_value
    dp = increment
    best_err = simulation(p)

    it = 0
    while sum(dp) > tol:
        print("It {}, best error = {}, p = {}, dp={}".format(it, best_err, p, dp))
        for i in range(len(p)):
            if dp[i] == 0: continue
            p[i] += dp[i]
            err = simulation(p)

            if err < best_err:
                best_err = err
                dp[i] *= 1.1
            else:
                p[i] -= 2 * dp[i]
                err = simulation(p)

                if err < best_err:
                    best_err = err
                    dp[i] /= conv_factor
                else:
                    p[i] += dp[i]
                    dp[i] *= conv_factor
        it += 1
    return p

# Automatic steering

In [4]:
class AutomaticThrottling: 
    throttle_gains = [0.8, 0, 0]
    max_steps = 3000
    
    
    def __init__(self):
        self.best_error = 1e6
        self.best_error_record = None

    def plot_data(self, error_data, title):
        fig, ax = plt.subplots(figsize=(12,5))
        ax.plot(error_data, 'b')
        
        if self.best_error_record is not None:
            ax.plot(self.best_error_record, 'r--')

        ax.spines['left'].set_position('zero')
        ax.spines['bottom'].set_position('zero')
        ax.spines['right'].set_color('none')
        ax.spines['top'].set_color('none')
        ax.set_ylim(-1, 1)
        ax.set_xlim(0, self.max_steps)
        ax.set_title(title)
        fig.savefig('report/test-%d.png' % int(time.time() * 1000))

        
    def __call__(self, gain_values):
        print("Running simulation with ", gain_values)
        steering = PidController(gains=gain_values, set_point=constant(0), max_measurement=3.0)
        throttle = PidController(gains=self.throttle_gains, set_point=constant(50))

        def stop_for_big_error(simulator):
            return steering.overflow_detected or (simulator.step_count > self.max_steps)

        run_simulator(throttle, steering, stop_for_big_error)
        error_record = steering.get_history().error
        error_metric = steering.get_error()
        
        if error_metric < self.best_error:
            self.best_error = error_metric
            self.best_error_record = error_record
        
        self.plot_data(error_record, str(gain_values))
        print("Error is: ", error_metric)
        return error_metric
    
[os.remove(f) for f in glob.glob('report/*.png')]

best_guess = twiddle(AutomaticThrottling(), 
                     initial_value=[0.07, 0, 0.07], 
                     increment=[0.04, 0.1, 0.05], 
                     tol=0.01)

print("Best guess is: ", best_guess)
clips = [mpy.ImageClip(fname, duration=0.5) for fname in sorted(glob.glob('report/*.png'))]
concat = mpy.concatenate_videoclips(clips)
concat.write_videofile('tuning.mp4', fps=10)

Running simulation with  [0.07, 0, 0.07]
Error is:  0.7938403326647965
It 0, best error = 0.7938403326647965, p = [0.07, 0, 0.07], dp=[0.04, 0.1, 0.05]
Running simulation with  [0.11000000000000001, 0, 0.07]
Error is:  0.40622706584151363
Running simulation with  [0.11000000000000001, 0.1, 0.07]
Error is:  1.3333585344880814
Running simulation with  [0.11000000000000001, -0.1, 0.07]
Error is:  230948.73153832686
Running simulation with  [0.11000000000000001, 0.0, 0.12000000000000001]
Error is:  0.3399194593057502
It 1, best error = 0.3399194593057502, p = [0.11000000000000001, 0.0, 0.12000000000000001], dp=[0.044000000000000004, 0.09000000000000001, 0.05500000000000001]
Running simulation with  [0.15400000000000003, 0.0, 0.12000000000000001]
Error is:  0.21930300942847109
Running simulation with  [0.15400000000000003, 0.09000000000000001, 0.12000000000000001]
Error is:  0.10887509585553992
Running simulation with  [0.15400000000000003, 0.09000000000000001, 0.17500000000000002]
Error is

100%|█████████▉| 1090/1091 [00:09<00:00, 110.92it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: tuning.mp4 



# Manual tuning

In [ ]:
best_guess = [0.247, 0.033, 0.105]
steering = PidController(gains=best_guess, set_point=constant(0), max_measurement=3.2)
throttle = PidController(gains=[0.8, 0, 0], set_point=constant(40))

run_simulator(throttle, steering, max_steps(1e5))
h = steering.get_history()